Analysis of different sampling methods to determine the optimal sampling strategy for the final OJO sample.

In [1]:
import pandas as pd

from dap_prinz_green_jobs.getters.ojo_getters import (
    get_large_ojo_job_title_sample,
    get_large_ojo_location_sample
)

from dap_prinz_green_jobs.getters.data_getters import load_s3_data
from dap_prinz_green_jobs import BUCKET_NAME, config, PROJECT_DIR

import altair as alt
import os
from datetime import datetime

In [2]:
alt.data_transformers.disable_max_rows()

random_seed = 42
sample_size = 1000000

#save graphs
today = datetime.today().strftime('%y%m%d')
graph_dir = str(PROJECT_DIR / f"outputs/figures/sampling_analysis/{today}/")

if not os.path.exists(graph_dir):
    print(f"Creating {graph_dir} directory")
    os.makedirs(graph_dir)
else:
    print(f"{graph_dir} directory already exists")

Creating /Users/india.kerlenesta/Projects/dap_green_jobs/dap_prinz_green_jobs/outputs/figures/sampling_analysis/231122 directory


0. Load data
- load current soc4 itl2 code large sample
- unique soc4 code to job title json file
- overall job locations data
- deduplicated ids data

In [3]:
#load SOC codes to job titles mapper
soc_job_titles = load_s3_data(BUCKET_NAME, "outputs/data/ojo_application/deduplicated_sample/jobtitles2soc4_production_true.json")

#load current sample information
current_ojo_sample_titles = get_large_ojo_job_title_sample()
current_ojo_sample_locations = get_large_ojo_location_sample()

#load full locations and titles data 
locations_data = pd.read_parquet(config["ojo_s3_file_locations"])
locations_data = locations_data.drop_duplicates()

all_titles = pd.read_parquet(config["ojo_s3_file_adverts_ojd_daps_extract"])
all_titles = all_titles.drop_duplicates(
        subset=all_titles.columns.difference(["created"])
    )

#load deduplicated ids 
deduplicated_ids = pd.read_csv(
        "s3://prinz-green-jobs/outputs/data/ojo_application/deduplicated_sample/deduplicated_job_ids.csv"
    )
deduplicated_ids_list = deduplicated_ids.id.to_list()

2023-11-22 10:23:55,486 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2023-11-22 10:23:57,755 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


## 1. sampling analysis

look into:
- **deduplication:** differences in deduplication between deduplicated ids and all_titles/locations
- **sampling:** location differences in sampling between random sample and current sample data 

In [4]:
# deduplication 
print(f"there are {len(deduplicated_ids_list)} unique, deduplicated ids")
print(f"there are {len(all_titles)} unique ids from the full ojo sample job titles dataset")
print(f"{round((len(deduplicated_ids_list) / len(all_titles))*100, 2)}% of job ads are duplicates")

# are we deduplicating incorrectly? looks like we're loosing a lot of job ids. We might need to re-run the deduplication script
# as we've collected more data since the projects start?

there are 3618729 unique, deduplicated ids
there are 7883041 unique ids from the full ojo sample job titles dataset
45.91% of job ads are duplicates


### randomly sample dataset + add soc4 codes to datasets

In [5]:
# let's look into sampling - first lets merge the current title and locations samples and add soc4 codes

sample_titles_locations = pd.merge(current_ojo_sample_titles, current_ojo_sample_locations, on='id')
sample_titles_locations['soc4_code'] = sample_titles_locations['job_title_raw'].map(soc_job_titles)

#now, let's take a random sample of 1,000,000 job titles from the full dataset and add soc4 codes
full_titles_locations = pd.merge(all_titles, locations_data, on='id')
full_titles_locations_sample = full_titles_locations.sample(n=sample_size, random_state=random_seed)
full_titles_locations_sample['soc4_code'] = full_titles_locations_sample['job_title_raw'].map(soc_job_titles)

#deduplicate the full dataset
deduped_full_titles_locations = full_titles_locations.query('id in @deduplicated_ids_list')
deduped_full_titles_locations_sample = deduped_full_titles_locations.sample(n=sample_size, random_state=random_seed)
deduped_full_titles_locations_sample['soc4_code'] = deduped_full_titles_locations_sample['job_title_raw'].map(soc_job_titles)


In [7]:
print(f"there are {sample_titles_locations.itl_2_code.nunique()} itl 2 codes in the oversampled dataset")
print(f"there are {full_titles_locations.itl_2_code.nunique()} itl 2 codes in the full dataset")
print(f"there are {full_titles_locations_sample.itl_2_code.nunique()} itl 2 codes in the randomly dataset")
print(f"there are {deduped_full_titles_locations_sample.itl_2_code.nunique()} itl 2 codes in the deduped randomly dataset")

print('')
print(f"there are {sample_titles_locations.itl_3_code.nunique()} itl 3 codes in the oversampled dataset")
print(f"there are {full_titles_locations.itl_3_code.nunique()} itl 3 codes in the full dataset")
print(f"there are {full_titles_locations_sample.itl_3_code.nunique()} itl 3 codes in the randomly dataset")
print(f"there are {deduped_full_titles_locations_sample.itl_3_code.nunique()} itl 3 codes in the deduped randomly dataset")


#looks like theres pretty much all itl 2/3 codes are represented in the over/full/and randomly sampled datasets

there are 37 itl 2 codes in the oversampled dataset
there are 37 itl 2 codes in the full dataset
there are 37 itl 2 codes in the randomly dataset
there are 37 itl 2 codes in the deduped randomly dataset

there are 158 itl 3 codes in the oversampled dataset
there are 159 itl 3 codes in the full dataset
there are 159 itl 3 codes in the randomly dataset
there are 159 itl 3 codes in the deduped randomly dataset


### Oversampled sample analysis

In [40]:
itl2_counts = sample_titles_locations.groupby('itl_2_name').size().reset_index().rename(columns={0:'count'})
itl2_counts['above_100'] = itl2_counts['count'] > 100

itl3_counts = sample_titles_locations.groupby('itl_3_name').size().reset_index().rename(columns={0:'count'})
itl3_counts['above_50'] = itl3_counts['count'] > 50
itl3_counts['above_100'] = itl3_counts['count'] > 100

print(f"{round(len(itl2_counts.query('above_100 == False')) / len(itl2_counts), 2)*100}% of itl 2 codes have less than 100 job ads")
print('')
print(f"{round(len(itl3_counts.query('above_50 == False')) / len(itl2_counts), 2)*100}% of itl 3 codes have less than 50 job ads")
print(f"{round(len(itl3_counts.query('above_100 == False')) / len(itl2_counts), 2)*100}% of itl 3 codes have less than 100 job ads")

2.0% of itl 2 codes have less than 100 job ads

56.99999999999999% of itl 3 codes have less than 50 job ads
71.0% of itl 3 codes have less than 100 job ads


In [41]:
#top itl codes in oversampled dataset
itl_counts = sample_titles_locations.groupby('itl_2_name').size().sort_values(ascending=False).head(10).reset_index().rename(columns={0:'count'})
itl_2_graph = alt.Chart(itl_counts, title=f"top locations at itl 2 level").mark_bar(opacity=0.2, color='red').encode(
    x=alt.X('count', title='Number of job ads'),
    #add label limit to be 50000
    y=alt.Y(f'itl_2_name', title='', sort='-x', axis=alt.Axis(labelLimit=50000)))

itl_counts = sample_titles_locations.groupby('itl_3_name').size().sort_values(ascending=False).head(10).reset_index().rename(columns={0:'count'})
itl_3_graph = alt.Chart(itl_counts, title=f"top locations at itl 3 level").mark_bar(opacity=0.2, color='blue').encode(
    x=alt.X('count', title='Number of job ads'),
    #add label limit to be 50000
    y=alt.Y(f'itl_3_name', title='', sort='-x', axis=alt.Axis(labelLimit=50000)))

itl_counts_graphs = itl_2_graph | itl_3_graph

In [42]:
soc4_itl2_counts = sample_titles_locations.groupby(['soc4_code', 'itl_2_code']).size().reset_index().rename(columns={0:'count'}).sort_values(by='count', ascending=True)
soc4_itl2_counts['soc4_itl2'] = soc4_itl2_counts['soc4_code'] + ' - ' + soc4_itl2_counts['itl_2_code']

soc4_itl2_counts['above_1'] = soc4_itl2_counts['count'] > 1
soc4_itl2_counts['above_5'] = soc4_itl2_counts['count'] > 5
soc4_itl2_counts['above_10'] = soc4_itl2_counts['count'] > 10
soc4_itl2_counts['above_50'] = soc4_itl2_counts['count'] > 50

above_1df = soc4_itl2_counts.above_1.value_counts().reset_index().rename(columns={'index':'above_1', 'above_1':'count'})
more_than1_graph = alt.Chart(above_1df).mark_bar().encode(
    y=alt.Y('above_1', title=''),
    x=alt.X('count', title='Number of job ads'),
    color=alt.Color('above_1', title='above 1', scale=alt.Scale(scheme='dark2'), legend=None)
).properties(
    title={
      "text": ["more than 1 job ad"], 
      "subtitle": [f"{(round(above_1df.query('above_1 == True')['count']/above_1df['count'].sum(), 2)*100).values[0]}% have more than 1 job ad"],
    }
)

above_5df = soc4_itl2_counts.above_5.value_counts().reset_index().rename(columns={'index':'above_5', 'above_5':'count'})
more_than5_graph = alt.Chart(above_5df).mark_bar().encode(
    y=alt.Y('above_5', title=''),
    x=alt.X('count', title='Number of job ads'),
    color=alt.Color('above_5', title='above 5', scale=alt.Scale(scheme='dark2'), legend=None)
).properties(
    title={
      "text": ["more than 5 job ads"], 
      "subtitle": [f"{(round(above_5df.query('above_5 == True')['count']/above_5df['count'].sum(), 2)*100).values[0]}% have more than 5 job ads"],
    }
)

above_10df = soc4_itl2_counts.above_10.value_counts().reset_index().rename(columns={'index':'above_10', 'above_10':'count'})
more_than10_graph = alt.Chart(above_10df).mark_bar().encode(
    y=alt.Y('above_10', title=''),
    x=alt.X('count', title='Number of job ads'),
    color=alt.Color('above_10', title='above 10', scale=alt.Scale(scheme='dark2'), legend=None)
).properties(
    title={
      "text": ["more than 10 job ads"], 
      "subtitle": [f"{(round(above_10df.query('above_10 == True')['count']/above_10df['count'].sum(), 2)*100).values[0]}% have more than 10 job ads"],
    }
)

above_50df = soc4_itl2_counts.above_50.value_counts().reset_index().rename(columns={'index':'above_50', 'above_50':'count'})
more_than50_graph = alt.Chart(above_50df).mark_bar().encode(
    y=alt.Y('above_50', title=''),
    x=alt.X('count', title='Number of job ads'),
    color=alt.Color('above_50', title='above 10', scale=alt.Scale(scheme='dark2'), legend=None)
).properties(
    title={
      "text": ["more than 50 job ads"], 
      "subtitle": [f"{(round(above_50df.query('above_50 == True')['count']/above_50df['count'].sum(), 2)*100).values[0]}% have more than 10 job ads"],
    }
)

oversample_charts = (itl_counts_graphs & (more_than1_graph | more_than5_graph) & (more_than10_graph | more_than50_graph)).properties(
    title={"text": "Sampling analysis for oversampled dataset of 1000000 deduplicated job ids"}
)

oversample_charts.save(f'{graph_dir}/oversampled_analysis.html')

### Randomly sampled analysis

In [43]:
itl2_counts = full_titles_locations_sample.groupby('itl_2_name').size().reset_index().rename(columns={0:'count'})
itl2_counts['above_100'] = itl2_counts['count'] > 100

itl3_counts = full_titles_locations_sample.groupby('itl_3_name').size().reset_index().rename(columns={0:'count'})
itl3_counts['above_50'] = itl3_counts['count'] > 50
itl3_counts['above_100'] = itl3_counts['count'] > 100

print(f"{round(len(itl2_counts.query('above_100 == False')) / len(itl2_counts), 2)*100}% of itl 2 codes have less than 100 job ads")
print('')
print(f"{round(len(itl3_counts.query('above_50 == False')) / len(itl2_counts), 2)*100}% of itl 3 codes have less than 50 job ads")
print(f"{round(len(itl3_counts.query('above_100 == False')) / len(itl2_counts), 2)*100}% of itl 3 codes have less than 100 job ads")

0.0% of itl 2 codes have less than 100 job ads

2.0% of itl 3 codes have less than 50 job ads
7.000000000000001% of itl 3 codes have less than 100 job ads


In [44]:
#top itl codes in oversampled dataset
itl_counts = full_titles_locations_sample.groupby(f'itl_2_name').size().sort_values(ascending=False).head(10).reset_index().rename(columns={0:'count'})
itl_2_graph = alt.Chart(itl_counts, title=f"top locations at itl 2 level").mark_bar(opacity=0.2, color='green').encode(
    x=alt.X('count', title='Number of job ads'),
    #add label limit to be 50000
    y=alt.Y(f'itl_2_name', title='', sort='-x', axis=alt.Axis(labelLimit=50000)))

itl_counts = full_titles_locations_sample.groupby(f'itl_3_name').size().sort_values(ascending=False).head(10).reset_index().rename(columns={0:'count'})
itl_3_graph = alt.Chart(itl_counts, title=f"top locations at itl 3 level").mark_bar(opacity=0.2, color='purple').encode(
    x=alt.X('count', title='Number of job ads'),
    #add label limit to be 50000
    y=alt.Y(f'itl_3_name', title='', sort='-x', axis=alt.Axis(labelLimit=50000)))

itl_counts_graphs = itl_2_graph | itl_3_graph

In [45]:
soc4_itl2_counts = full_titles_locations_sample.groupby(['soc4_code', 'itl_2_code']).size().reset_index().rename(columns={0:'count'}).sort_values(by='count', ascending=True)
soc4_itl2_counts['soc4_itl2'] = soc4_itl2_counts['soc4_code'] + ' - ' + soc4_itl2_counts['itl_2_code']

soc4_itl2_counts['above_1'] = soc4_itl2_counts['count'] > 1
soc4_itl2_counts['above_5'] = soc4_itl2_counts['count'] > 5
soc4_itl2_counts['above_10'] = soc4_itl2_counts['count'] > 10
soc4_itl2_counts['above_50'] = soc4_itl2_counts['count'] > 50

above_1df = soc4_itl2_counts.above_1.value_counts().reset_index().rename(columns={'index':'above_1', 'above_1':'count'})
more_than1_graph = alt.Chart(above_1df).mark_bar().encode(
    y=alt.Y('above_1', title=''),
    x=alt.X('count', title='Number of job ads'),
    color=alt.Color('above_1', title='above 1', scale=alt.Scale(scheme='dark2'), legend=None)
).properties(
    title={
      "text": ["more than 1 job ad"], 
      "subtitle": [f"{(round(above_1df.query('above_1 == True')['count']/above_1df['count'].sum(), 2)*100).values[0]}% have more than 1 job ad"],
    }
)

above_5df = soc4_itl2_counts.above_5.value_counts().reset_index().rename(columns={'index':'above_5', 'above_5':'count'})
more_than5_graph = alt.Chart(above_5df).mark_bar().encode(
    y=alt.Y('above_5', title=''),
    x=alt.X('count', title='Number of job ads'),
    color=alt.Color('above_5', title='above 5', scale=alt.Scale(scheme='dark2'), legend=None)
).properties(
    title={
      "text": ["more than 5 job ads"], 
      "subtitle": [f"{(round(above_5df.query('above_5 == True')['count']/above_5df['count'].sum(), 2)*100).values[0]}% have more than 5 job ads"],
    }
)

above_10df = soc4_itl2_counts.above_10.value_counts().reset_index().rename(columns={'index':'above_10', 'above_10':'count'})
more_than10_graph = alt.Chart(above_10df).mark_bar().encode(
    y=alt.Y('above_10', title=''),
    x=alt.X('count', title='Number of job ads'),
    color=alt.Color('above_10', title='above 10', scale=alt.Scale(scheme='dark2'), legend=None)
).properties(
    title={
      "text": ["more than 10 job ads"], 
      "subtitle": [f"{(round(above_10df.query('above_10 == True')['count']/above_10df['count'].sum(), 2)*100).values[0]}% have more than 10 job ads"],
    }
)

above_50df = soc4_itl2_counts.above_50.value_counts().reset_index().rename(columns={'index':'above_50', 'above_50':'count'})
more_than50_graph = alt.Chart(above_50df).mark_bar().encode(
    y=alt.Y('above_50', title=''),
    x=alt.X('count', title='Number of job ads'),
    color=alt.Color('above_50', title='above 10', scale=alt.Scale(scheme='dark2'), legend=None)
).properties(
    title={
      "text": ["more than 50 job ads"], 
      "subtitle": [f"{(round(above_50df.query('above_50 == True')['count']/above_50df['count'].sum(), 2)*100).values[0]}% have more than 10 job ads"],
    }
)

random_sample_charts = (itl_counts_graphs & (more_than1_graph | more_than5_graph) & (more_than10_graph | more_than50_graph)).properties(
    title={"text": "Sampling analysis for randomly sampled dataset of 1000000 job ids"}
)

random_sample_charts.save(f'{graph_dir}/random_sample_analysis.html')

### Dedupled randomly sampled analysis 

In [46]:
itl2_counts = deduped_full_titles_locations_sample.groupby('itl_2_name').size().reset_index().rename(columns={0:'count'})
itl2_counts['above_100'] = itl2_counts['count'] > 100

itl3_counts = deduped_full_titles_locations_sample.groupby('itl_3_name').size().reset_index().rename(columns={0:'count'})
itl3_counts['above_50'] = itl3_counts['count'] > 50
itl3_counts['above_100'] = itl3_counts['count'] > 100

print(f"{round(len(itl2_counts.query('above_100 == False')) / len(itl2_counts), 2)*100}% of itl 2 codes have less than 100 job ads")
print('')
print(f"{round(len(itl3_counts.query('above_50 == False')) / len(itl2_counts), 2)*100}% of itl 3 codes have less than 50 job ads")
print(f"{round(len(itl3_counts.query('above_100 == False')) / len(itl2_counts), 2)*100}% of itl 3 codes have less than 100 job ads")

0.0% of itl 2 codes have less than 100 job ads

2.0% of itl 3 codes have less than 50 job ads
12.0% of itl 3 codes have less than 100 job ads


In [47]:
#top itl codes in oversampled dataset
itl_counts = deduped_full_titles_locations_sample.groupby(f'itl_2_name').size().sort_values(ascending=False).head(10).reset_index().rename(columns={0:'count'})
itl_2_graph = alt.Chart(itl_counts, title=f"top locations at itl 2 level").mark_bar(opacity=0.2, color='green').encode(
    x=alt.X('count', title='Number of job ads'),
    #add label limit to be 50000
    y=alt.Y(f'itl_2_name', title='', sort='-x', axis=alt.Axis(labelLimit=50000)))

itl_counts = deduped_full_titles_locations_sample.groupby(f'itl_3_name').size().sort_values(ascending=False).head(10).reset_index().rename(columns={0:'count'})
itl_3_graph = alt.Chart(itl_counts, title=f"top locations at itl 3 level").mark_bar(opacity=0.2, color='purple').encode(
    x=alt.X('count', title='Number of job ads'),
    #add label limit to be 50000
    y=alt.Y(f'itl_3_name', title='', sort='-x', axis=alt.Axis(labelLimit=50000)))

itl_counts_graphs = itl_2_graph | itl_3_graph

In [48]:
soc4_itl2_counts = deduped_full_titles_locations_sample.groupby(['soc4_code', 'itl_2_code']).size().reset_index().rename(columns={0:'count'}).sort_values(by='count', ascending=True)
soc4_itl2_counts['soc4_itl2'] = soc4_itl2_counts['soc4_code'] + ' - ' + soc4_itl2_counts['itl_2_code']

soc4_itl2_counts['above_1'] = soc4_itl2_counts['count'] > 1
soc4_itl2_counts['above_5'] = soc4_itl2_counts['count'] > 5
soc4_itl2_counts['above_10'] = soc4_itl2_counts['count'] > 10
soc4_itl2_counts['above_50'] = soc4_itl2_counts['count'] > 50

above_1df = soc4_itl2_counts.above_1.value_counts().reset_index().rename(columns={'index':'above_1', 'above_1':'count'})
more_than1_graph = alt.Chart(above_1df).mark_bar().encode(
    y=alt.Y('above_1', title=''),
    x=alt.X('count', title='Number of job ads'),
    color=alt.Color('above_1', title='above 1', scale=alt.Scale(scheme='dark2'), legend=None)
).properties(
    title={
      "text": ["more than 1 job ad"], 
      "subtitle": [f"{(round(above_1df.query('above_1 == True')['count']/above_1df['count'].sum(), 2)*100).values[0]}% have more than 1 job ad"],
    }
)

above_5df = soc4_itl2_counts.above_5.value_counts().reset_index().rename(columns={'index':'above_5', 'above_5':'count'})
more_than5_graph = alt.Chart(above_5df).mark_bar().encode(
    y=alt.Y('above_5', title=''),
    x=alt.X('count', title='Number of job ads'),
    color=alt.Color('above_5', title='above 5', scale=alt.Scale(scheme='dark2'), legend=None)
).properties(
    title={
      "text": ["more than 5 job ads"], 
      "subtitle": [f"{(round(above_5df.query('above_5 == True')['count']/above_5df['count'].sum(), 2)*100).values[0]}% have more than 5 job ads"],
    }
)

above_10df = soc4_itl2_counts.above_10.value_counts().reset_index().rename(columns={'index':'above_10', 'above_10':'count'})
more_than10_graph = alt.Chart(above_10df).mark_bar().encode(
    y=alt.Y('above_10', title=''),
    x=alt.X('count', title='Number of job ads'),
    color=alt.Color('above_10', title='above 10', scale=alt.Scale(scheme='dark2'), legend=None)
).properties(
    title={
      "text": ["more than 10 job ads"], 
      "subtitle": [f"{(round(above_10df.query('above_10 == True')['count']/above_10df['count'].sum(), 2)*100).values[0]}% have more than 10 job ads"],
    }
)

above_50df = soc4_itl2_counts.above_50.value_counts().reset_index().rename(columns={'index':'above_50', 'above_50':'count'})
more_than50_graph = alt.Chart(above_50df).mark_bar().encode(
    y=alt.Y('above_50', title=''),
    x=alt.X('count', title='Number of job ads'),
    color=alt.Color('above_50', title='above 10', scale=alt.Scale(scheme='dark2'), legend=None)
).properties(
    title={
      "text": ["more than 50 job ads"], 
      "subtitle": [f"{(round(above_50df.query('above_50 == True')['count']/above_50df['count'].sum(), 2)*100).values[0]}% have more than 10 job ads"],
    }
)

deduped_random_sample_charts = (itl_counts_graphs & (more_than1_graph | more_than5_graph) & (more_than10_graph | more_than50_graph)).properties(
    title={"text": "Sampling analysis for randomly sampled dataset of 1000000 deduplicated job ids"}
)

deduped_random_sample_charts.save(f'{graph_dir}/random_deduped_sample_analysis.html')

In [49]:
#save all of them together! 

(deduped_random_sample_charts | random_sample_charts | oversample_charts).save(f'{graph_dir}/sampling_analysis.html')

Takeaways:
- the **weighted sample** is quite skewed at the itl3 level i.e. most regions have fewer than 100 job ads and half have fewer than 50 job ads.
- the deduplication method does get rid of almost half of the job adverts: i wonder if its worth re-running the script as I know jack's been collecting job ads? I don't think that will change the results drastically but could be good to be updated
- we would still need to oversample in a few regions if we move forward with a totally random sample 